In [47]:
import gc
import time
import numpy as np
import pandas as pd

In [48]:
test = pd.read_csv('databases/test.tsv', sep = '\t')

In [49]:
train = pd.read_csv('databases/train.tsv', sep = '\t')

In [50]:
test.head()

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
0,0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined..."
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...


In [51]:
train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [52]:
train.price.describe()

count    29999.000000
mean        26.953582
std         40.394998
min          0.000000
25%         10.000000
50%         17.000000
75%         29.000000
max       1506.000000
Name: price, dtype: float64

In [59]:
!pip install lightgbm

In [60]:
import lightgbm as lgb

In [54]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelBinarizer

In [55]:
from scipy.sparse import csr_matrix, hstack

In [105]:
Num_Brand = 5000
Num_Cat = 1000
Min_Df = 5
Max_Feat_Item_Des = 30000

In [65]:
#Helper functions for LightGBM

In [5]:
#Finds missing values within dataset and replaces with 'NA'
def fill_NA_values(dataset):
    dataset['category_name'].fillna(value='NA', inplace=True)
    dataset['brand_name'].fillna(value='NA', inplace=True)
    dataset['item_description'].fillna(value='NA', inplace=True)

In [57]:
def cutting(dataset):
    #.value_counts() returns the relative frequencies of unique values.
    #.loc - Access a group of rows and columns by label(s) or a boolean array.
    #lambda - if TRUE return 1, else return 0
    pop_brand = dataset['brand_name'].value_counts().loc[lambda x: x.index != 'missing'].index[:Num_Brand]
    dataset.loc[~dataset['brand_name'].isin(pop_brand), 'brand_name'] = 'missing'
    pop_category = dataset['category_name']
    dataset.loc[lambda x: x.index != 'missing'].index[:Num_Brand]

In [4]:
#Puts it into binary and then places into a matrix
def to_categorical(dataset):
    dataset['category_name'] = dataset['category_name'].astype('category')
    dataset['brand_name'] = dataset['brand_name'].astype('category')
    dataset['item_condition_id'] = dataset['item_condition_id'].astype('category')

In [101]:
test = pd.read_csv('databases/test.tsv', engine='c', sep='\t')
train = pd.read_csv('databases/train.tsv', engine='c', sep='\t')
print('Train shape: ', train.shape)
print('Test shape: ', test.shape)

Train shape:  (29999, 8)
Test shape:  (29999, 7)


In [28]:
#Merging the train and test data
nrow_train = train.shape[0]
y = np.log1p(train["price"])
merge: pd.DataFrame = pd.concat([train, test])
submission: pd.DataFrame = test[['test_id']]

In [67]:
merge.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,test_id
0,0.0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NA,10.0,1,No description yet,NaN
1,1.0,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,NaN
2,2.0,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,NaN
3,3.0,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NA,35.0,1,New with tags. Leather horses. Retail for [rm]...,NaN
4,4.0,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NA,44.0,0,Complete with certificate of authenticity,NaN


In [58]:
#calling function 'fill_NA_values'
fill_NA_values(merge)
#calling function 'cutting'
cutting(merge)
#calling function 'merge'
to_categorical(merge)

In [66]:
merge.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,test_id
0,0.0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NA,10.0,1,No description yet,NaN
1,1.0,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,NaN
2,2.0,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,NaN
3,3.0,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NA,35.0,1,New with tags. Leather horses. Retail for [rm]...,NaN
4,4.0,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NA,44.0,0,Complete with certificate of authenticity,NaN


In [80]:
#CountVectorizer - Converts raw text to smaller units of text/numerical data

#Count vectorize category name columns.
cv = CountVectorizer(min_df=Min_Df)
X_name = cv.fit_transform(merge['name'])
#Count vectorize category name columns.
cv = CountVectorizer()
X_category = cv.fit_transform(merge['category_name'])
#Highlights words with high frequency score or considered 'intersting'
tv = TfidfVectorizer(max_features=Max_Feat_Item_Des,
                         ngram_range=(1, 3),
                         stop_words='english')
X_description = tv.fit_transform(merge['item_description'])

In [72]:
#Label Binarizer is an SciKit Learn class that accepts Categorical data as input and returns an Numpy array.
#True if the returned array from transform is desired to be in sparse CSR format.
lb = LabelBinarizer(sparse_output=True)
#fit_transform() is used on the training data so that we can scale the training data and also learn the scaling parameters of that data. 
X_brand = lb.fit_transform(merge['brand_name'])

In [73]:
#Creating sparse matrix which includes dummy variables for item condition id and shipping columns
X_dummies = csr_matrix(pd.get_dummies(merge[['item_condition_id', 'shipping']],
                                          sparse=True).values)

In [81]:
#Combining everything together
sparse_merge = hstack((X_dummies, X_description, X_brand, X_category, X_name)).tocsr()

In [82]:
#Separating train and test data from the sparse merge
X = sparse_merge[:nrow_train]
X_test = sparse_merge[nrow_train:]

In [84]:
#Create dataset for lightgbm
d_train = lgb.Dataset(X, label=y)

In [107]:
#learning rate is a tuning parameter that determines the impact of each tree on the final outcome
#regression problem
#use max_depth to avoid overfitting
#num_leaves: larger num_leaves increases accuracy in the training set and also the chance of getting hurt by overfitting
#verbosity controls the level of LightGBM's verbosity. We set to -1 because we don't want detailed logging information.
#RMSE is the equation being used
params = {
        'learning_rate': 0.75,
        'application': 'regression',
        'max_depth': 3,
        'num_leaves': 100,
        'verbosity': -1,
        'metric': 'RMSE',
    }

In [88]:
#Training a model requires a parameter list and data set. And training will take a while.
gbm = lgb.train(params, train_set=d_train, num_boost_round=3200, verbose_eval=100)

In [91]:
#Prediction
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

In [106]:
#RMSE Evaluation
from sklearn.metrics import mean_squared_error

print('The rmse of prediction is:', mean_squared_error(y, y_pred) ** 0.5)

The rmse of prediction is: 0.9915725061151675
